In [ ]:
!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# 資料預處理

## Before dataloader

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("nyu-mll/glue", "rte", split="train")
valid_dataset = load_dataset("nyu-mll/glue", "rte", split="validation")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0])

{'sentence1': 'No Weapons of Mass Destruction Found in Iraq Yet.', 'sentence2': 'Weapons of Mass Destruction Found in Iraq.', 'label': 1, 'idx': 0}


### Naive method


In [ ]:
# 自定義資料處理方式
def read_data(dataset):
    sentence1=[]
    sentence2=[]
    label=[]
    idx=[]
    for text in dataset:
        sentence1.append(text['sentence1'])
        sentence2.append(text['sentence2'])
        label.append(text['label'])
        idx.append(text['idx'])
    return sentence1,sentence2,label,idx

In [ ]:
train_sen1, train_sen2, trainlabel, trainidx = read_data(train_dataset)
valid_sen1, valid_sen2, validlabel, valididx = read_data(valid_dataset)

In [ ]:
# 1.當資料量大時，RAM 會不夠  2.要自己加label
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
valid_encodings = tokenizer(valid_sen1, valid_sen2, truncation=True, padding=True)

In [ ]:
def add_targets(encodings,label):
    encodings.update({'label':label})
add_targets(train_encodings,trainlabel)
add_targets(valid_encodings,validlabel)

In [ ]:
print('train 資料集數量= ',len(train_dataset['label']))
print('valid 資料集數量= ',len(valid_dataset['label']))

train 資料集數量=  2490
valid 資料集數量=  277


In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])

Encoding(num_tokens=289, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
# 自定義 dataset for dataloader
import torch
class HuggingFaceDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    # 知道樣本數 => 知道迭代次數
    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    # 取樣並轉為 tensor
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}

# 創建 Dataset
train_encodings = HuggingFaceDataset(train_encodings)
valid_encodings = HuggingFaceDataset(valid_encodings)

In [ ]:
train_encodings[0]

{'input_ids': tensor([ 101, 2053, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 2664, 1012,  102,
         4255, 1997, 3742, 6215, 2179, 1999, 5712, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 


### map + set_format

In [ ]:
# 自定義資料處理方式
def tokenize_function(example):
    return tokenizer(
        example["sentence1"],
        example["sentence2"],
        padding="max_length",
        truncation=True
    )

In [ ]:
# RAM 裡面只會有部分資料，部分結果會轉到 disk
train_data = train_dataset.map(tokenize_function, batched=True)
valid_data = valid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

In [ ]:
train_data[0].keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])
valid_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])

In [ ]:
train_data[0]

{'label': tensor(1),
 'input_ids': tensor([ 101, 2053, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 2664, 1012,  102,
         4255, 1997, 3742, 6215, 2179, 1999, 5712, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 

## Dataloader 設定

In [ ]:
import logging
import datasets
from datasets import load_dataset
from evaluate import load
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math
import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

In [ ]:
train_batch_size = 12
valid_batch_size = 12
num_train_epochs = 10

In [ ]:
# 讓 padding, truncation 可以正確運作
data_collator = default_data_collator

train_dataloader = DataLoader(train_data, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
valid_dataloader = DataLoader(valid_data, collate_fn=data_collator, batch_size=valid_batch_size)

In [ ]:
print(len(train_dataloader))
print(len(train_data))

208
2490


# Optimizer 、Learning rate 、Scheduler 設定

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
learning_rate=3e-5
gradient_accumulation_steps = 1   # 設定幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler 動態調整 learning rate
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 2080


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, valid_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, valid_dataloader
)

# accuracy, recall, F1-score
metric = load("accuracy")

In [ ]:
# prompt: 掛載雲端硬碟

from google.colab import drive
drive.mount('/content/drive')
cd /content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
import os

# Create the 'model' directory if it doesn't exist
model_dir = "model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Directory '{model_dir}' created successfully.")
else:
    print(f"Directory '{model_dir}' already exists.")

Directory 'model' created successfully.


# Training

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = 'model'  # your folder


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break

  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(valid_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  print(f"epoch {epoch}: {eval_metric}")
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'new_epoch_' + str(epoch), save_function=accelerator.save)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/208 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 0.6937819123268127}
{'epoch': 0, 'step': 50, 'loss': 0.6069858074188232}
{'epoch': 0, 'step': 100, 'loss': 0.691882848739624}
{'epoch': 0, 'step': 150, 'loss': 0.6816840171813965}
{'epoch': 0, 'step': 200, 'loss': 0.8180441856384277}


eval Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

epoch 0: {'accuracy': 0.6750902527075813}


...

epoch 8: {'accuracy': 0.6823104693140795}


Iteration:   0%|          | 0/208 [00:00<?, ?it/s]

{'epoch': 9, 'step': 0, 'loss': 0.0010517745977267623}
{'epoch': 9, 'step': 50, 'loss': 0.00825080182403326}
{'epoch': 9, 'step': 100, 'loss': 0.001313007902354002}
{'epoch': 9, 'step': 150, 'loss': 0.001397794927470386}
{'epoch': 9, 'step': 200, 'loss': 0.0029796764720231295}


eval Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

epoch 9: {'accuracy': 0.7075812274368231}


In [ ]:
print(best_epoch)

{'epoch': 5, 'acc': 0.7292418772563177}


# Inference

In [ ]:
# rte task:假設 (sentence2) 能否從前提 (sentence1) 推導出來, 能 (entailment), 不能 (not_entailment)
test_sentence1 = "Mangla was summoned after Madhumita's sister Nidhi Shukla, who was the first witness in the case."
test_sentence2 = "Shukla is related to Mangla."
label = "not_entailment"

model.eval()
input = tokenizer(test_sentence1, test_sentence2, return_tensors="pt", padding=True)
input = {k: v.to(device) for k, v in input.items()}
output = model(**input)
logits = output.logits
predicted_class_id = logits.argmax().item()

if predicted_class_id == 1:
    print("not_entailment")
else:
    print("entailment")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
